## Climate Data


Climate Data for this analysis was sourced from the Northwest Knowledge Multivariate Adaptice Consttucted (MACA) Datasets, version 4 (https://climate.northwestknowledge.net/MACA/index.php)

The MACA datasets uses a statistical downscaling method which utilizes a training dataset (for example, a meteorological observation data) to remove historical biases and match spatial patterns in climate model output.  It contains information about the 4.5 and 8.5 Representative Concentration Pathways for different greenhouse gases emissions scenarios up to the year 2100 for variable such as temperature, precipitation, humidity, wind, and solar radiation, allowing for robust analysis for modeling future climate effects. 

> Source 
>
>Climate forcings in the MACAv2-METDATA were drawn from a statistical downscaling of global climate model (GCM) data from the Coupled Model Intercomparison Project 5 (CMIP5, Taylor et al. 2010) utilizing a modification of the Multivariate Adaptive Constructed Analogs (MACA, Abatzoglou and Brown, 2012) method with the METDATA(Abatzoglou, 2011) observational dataset as training data. http://onlinelibrary.wiley.com/doi/10.1002/joc.2312/full


In [1]:
# retrieve stored variables 
%store -r habitat_data_dir teton_gdf rmnp_gdf aoi_gdf

In [2]:
#import libraries 

import os
import pathlib
from glob import glob

import pandas as pd
import earthaccess
import geopandas as gpd
import xrspatial
import rioxarray as rxr
import rioxarray.merge as rxrmerge
import matplotlib.pyplot as plt
import cartopy.crs as ccrs # CRSs
import requests
import xarray as xr

In [3]:
# define functions 
def convert_longitude(longitude):
    """Convert longitude from range 0-360 to -180 to 180 """
    return (longitude-360) if longitude > 180 else longitude



In [4]:
# Set required data directories 
climate_data_dir = os.path.join(habitat_data_dir,'climate')
os.makedirs(climate_data_dir, exist_ok=True)

In [21]:
#format URL 
maca_da_list = []

for site_name, site_gdf in {'teton': teton_gdf}.items():
    for variable in ['tasmin']:
        for start_year in [2006, 2091]:
            end_year = start_year + 4
            maca_url = (
              'http://thredds.northwestknowledge.net:8080'
              '/thredds/dodsC/MACAV2/BNU-ESM'
              '/macav2metdata_'
              #Varabable tasmin, daily average min temperature 
              f'{variable}_BNU-ESM_r1i1p1'
              #climate scenario 
              '_rcp85'
              #years, in 5 years chunks
              f'_{start_year}_{end_year}'
              #monthly or daily 
              '_CONUS_monthly'
              #file format, need to open with xarray 
              '.nc'
            )
            print(maca_url)
            #sees if file is downloaded already, if not goes to clip and trim file before saving it
            filepath= os.path.join(climate_data_dir,f'{site_name}_{variable}_{start_year}_{end_year}.nc')
            if not os.path.exists(filepath):
                # set coordinates from 0-360 longitute to -180 to 180
                maca_da = xr.open_dataset(maca_url, engine="netcdf4").squeeze().air_temperature
                maca_da = maca_da.assign_coords(
                    lon =('lon', [convert_longitude(lon) for lon in maca_da.lon.values]))
                #clip based on input gdf
                bounds = site_gdf.to_crs(maca_da.rio.crs).total_bounds
                maca_da = maca_da.rio.set_spatial_dims(x_dim='lon', y_dim='lat')
                maca_da = maca_da.rio.clip_box(*bounds)
                #save to file path 
                filepath= os.path.join(climate_data_dir,f'{site_name}_{variable}_{start_year}_{end_year}.nc')
                maca_da.to_netcdf(filepath)
            else:
                maca_da = xr.open_dataset(filepath, engine="netcdf4").squeeze().air_temperature
            #append files 
            maca_da_list.append(dict(
                site_name=site_name,
                variable = variable,
                start_year = start_year,
                da=maca_da))

pd.DataFrame(maca_da_list) 


http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/BNU-ESM/macav2metdata_tasmin_BNU-ESM_r1i1p1_rcp85_2006_2010_CONUS_monthly.nc
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/BNU-ESM/macav2metdata_tasmin_BNU-ESM_r1i1p1_rcp85_2091_2095_CONUS_monthly.nc


,site_name,variable,start_year,da
0,teton,tasmin,2006,[[[<xarray.DataArray 'air_temperature' ()> Siz...
1,teton,tasmin,2091,[[[<xarray.DataArray 'air_temperature' ()> Siz...
